### Variáveis fundamentais

In [8454]:
temporada = "2025"
campeonato = "Libertadores"
pais = "América do Sul"

# Script

In [8455]:
# Caminho para salvar o arquivo
caminho = caminho = r"C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx"

### Bibliotecas

In [8456]:
import pandas as pd
import mysql.connector
import numpy as np
import clubes
import importlib
import os

### Funções

In [8457]:
# Função para excluir a coluna "player_id"
def eliminar_primeira_coluna(df: pd.DataFrame) -> pd.DataFrame:
    if df.shape[1] == 0:
        return df.copy()
    return df.drop(df.columns[0], axis=1)

In [8458]:
# Função para salvar o DF no Excel
def salvar_df_no_excel(df: pd.DataFrame, caminho: str, aba: str, index: bool = False) -> None:
    # Garante que a pasta exista (se houver pasta no caminho)
    pasta = os.path.dirname(caminho)
    if pasta:
        os.makedirs(pasta, exist_ok=True)

    if os.path.exists(caminho):
        # arquivo já existe -> substituir a aba
        with pd.ExcelWriter(
            caminho,
            engine="openpyxl",
            mode="a",
            if_sheet_exists="replace",
        ) as writer:
            df.to_excel(writer, sheet_name=aba, index=index)
    else:
        # arquivo novo
        with pd.ExcelWriter(
            caminho,
            engine="openpyxl",
            mode="w",
        ) as writer:
            df.to_excel(writer, sheet_name=aba, index=index)

    print(f"Arquivo salvo em: {caminho} | Aba: {aba}")

## Geral

### Consulta e criação do DataFrame

Carregando a consulta

In [8459]:
config = {
    "host": "127.0.0.1",
    "user": "admin",
    "password": "1234",
    "database": "bet_dados"
}

In [8460]:
conn = mysql.connector.connect(**config)

In [8461]:
query = f"""
SELECT
  data,
  hora,
  casa,
  gols_casa,
  gols_fora,
  fora,
  odd_casa,
  odd_empate,
  odd_fora,
  CASE
    WHEN gols_casa > gols_fora THEN casa
    WHEN gols_fora > gols_casa THEN fora
    ELSE 'Empate'
  END AS vencedor,
  CASE
    WHEN gols_casa > gols_fora THEN fora
    WHEN gols_fora > gols_casa THEN casa
    ELSE 'Empate'
  END AS perdedor,
  CASE
    WHEN gols_casa > gols_fora THEN odd_casa
    WHEN gols_fora > gols_casa THEN odd_fora
    ELSE 0
  END AS odd_vencedora,
  CASE
	WHEN gols_casa > gols_fora THEN 3
    WHEN gols_casa = gols_fora THEN 1
    ELSE 0
  END AS pontos_casa,
  CASE
	WHEN gols_casa < gols_fora THEN 3
    WHEN gols_casa = gols_fora THEN 1
    ELSE 0
  END AS pontos_fora
FROM partidas
WHERE temporada = '{temporada}'
  AND campeonato = '{campeonato}'
  AND pais = '{pais}';
"""

In [8462]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_30808\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [8463]:
conn.close()

In [8464]:
df.head()

,data,hora,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,perdedor,odd_vencedora,pontos_casa,pontos_fora
0,2025-02-04,21:30,Monagas,2,0,Defensor,3.00,3.00,2.55,Monagas,Defensor,3.00,3,0
1,2025-02-05,21:30,Nacional/URU,1,1,Alianza Lima,2.20,3.00,3.25,Empate,Empate,0.00,1,1
2,2025-02-06,21:30,Blooming,3,2,El Nacional,2.55,3.20,2.60,Blooming,El Nacional,2.55,3,0
3,2025-02-11,21:30,Defensor,0,2,Monagas,1.42,4.33,8.00,Monagas,Defensor,8.00,0,3
4,2025-02-12,21:30,Alianza Lima,3,1,Nacional/URU,1.57,3.40,6.00,Alianza Lima,Nacional/URU,1.57,3,0


### Desempenho dos Clubes

In [8465]:
desempenho = df.copy()

Jogos

In [8466]:
jogos = desempenho.shape[0]

jogos

150

Vitórias da Casa

In [8467]:
vitorias_casa = desempenho[desempenho["vencedor"] == desempenho["casa"]].shape[0]

vitorias_casa

74

% Vitórias da Casa

In [8468]:
prob_vitorias_casa = round((vitorias_casa / jogos) * 100, 2)

prob_vitorias_casa

49.33

Odd Média da Casa

In [8469]:
odd_media_casa = desempenho["odd_casa"].mean().round(2)

odd_media_casa

2.36

Odd Média Quando o Time de Casa Venceu

In [8470]:
odd_media_vitoria_casa = desempenho[desempenho["vencedor"] == desempenho["casa"]]["odd_casa"].mean().round(2)

odd_media_vitoria_casa

1.88

Gols dos Times da Casa

In [8471]:
gols_casa = desempenho["gols_casa"].sum()

gols_casa

213

Média de Gols da Casa

In [8472]:
media_gols_casa = round(gols_casa / jogos, 2)

media_gols_casa

1.42

Empates

In [8473]:
empates = desempenho[desempenho["vencedor"] == "Empate"].shape[0]

empates

37

Vitórias do Visitante

In [8474]:
vitorias_fora = desempenho[desempenho["vencedor"] == desempenho["fora"]].shape[0]

vitorias_fora

39

% Vitórias do Visitante

In [8475]:
prob_vitorias_fora = round((vitorias_fora / jogos) * 100, 2)

prob_vitorias_fora

26.0

Odd Média do Visitante

In [8476]:
odd_media_fora = desempenho["odd_fora"].mean().round(2)

odd_media_fora

5.81

Odd Média Quando o Time Visitante Venceu

In [8477]:
odd_media_vitoria_fora = desempenho[desempenho["vencedor"] == desempenho["fora"]]["odd_fora"].mean().round(2)

odd_media_vitoria_fora

3.57

Gols dos Times Visitantes

In [8478]:
gols_fora = desempenho["gols_fora"].sum()

gols_fora

147

Média de Gols dos Visitantes

In [8479]:
media_gols_fora = round(gols_fora / jogos, 2)

media_gols_fora

0.98

Total de gols

In [8480]:
gols = (desempenho["gols_casa"] + desempenho["gols_fora"]).sum()

gols

360

Média de gols

In [8481]:
media_gols = round(gols / jogos, 2)

media_gols

2.4

Odd Média do Vencedor

In [8482]:
odd_media_vencedor = round((odd_media_vitoria_casa + odd_media_vitoria_fora) / 2, 2)

odd_media_vencedor

2.72

#### Montando o DataFrame do time da casa

In [8483]:
dados = {
    "Jogos": jogos,
    "Vitórias do mandante": vitorias_casa,
    "Probabilidade de vitória do mandante": prob_vitorias_casa,
    "Odd média do time mandante": odd_media_casa,
    "Odd média quando o mandante venceu": odd_media_vitoria_casa,
    "Gols do mandante": gols_casa,
    "Média de gols do mandante": media_gols_casa,
    "Empates": empates,
    "Vitórias do visitante": vitorias_fora,
    "Probabilidade de vitória do visitante": prob_vitorias_fora,
    "Odd média do time visitante": odd_media_fora,
    "Odd média quando o visitante venceu": odd_media_vitoria_fora,
    "Gols do visitante": gols_fora,
    "Média de gols do visitante": media_gols_fora,
    "Total de gols": gols,
    "Média de gols geral": media_gols,
    "Odd média dos vencedores": odd_media_vencedor
}

In [8484]:
# --- ordem e checagem ---
ordem = [
    "Jogos", "Vitórias do mandante", "Probabilidade de vitória do mandante",
    "Odd média do time mandante", "Odd média quando o mandante venceu",
    "Gols do mandante", "Média de gols do mandante", "Empates",
    "Vitórias do visitante", "Probabilidade de vitória do visitante",
    "Odd média do time visitante", "Odd média quando o visitante venceu",
    "Gols do visitante", "Média de gols do visitante", "Total de gols",
    "Média de gols geral", "Odd média dos vencedores"
]

faltando = [k for k in ordem if k not in dados]
if faltando:
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")

dados_ordenados = {k: dados[k] for k in ordem}

df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=[campeonato]
)

# --- quais devem ser float com 2 casas ---
variaveis_float = {
    "Probabilidade de vitória do mandante",
    "Odd média do time mandante",
    "Odd média quando o mandante venceu",
    "Média de gols do mandante",
    "Probabilidade de vitória do visitante",
    "Odd média do time visitante",
    "Odd média quando o visitante venceu",
    "Média de gols do visitante",
    "Média de gols geral",
    "Odd média dos vencedores",
}

def _fmt(nome, valor):
    # trata NaN/None
    if pd.isna(valor):
        return ""
    # se estiver marcado como float → 2 casas
    if nome in variaveis_float:
        try:
            return f"{float(valor):.2f}"
        except Exception:
            return str(valor)
    # caso contrário, formata como inteiro (arredondado)
    try:
        return f"{int(round(float(valor)))}"
    except Exception:
        return str(valor)

df_final_partidas[campeonato] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[campeonato])
]

# Índice vira coluna "Informações"
tabela = df_final_partidas.reset_index().rename(columns={"index": "Informações"})


In [8485]:
tabela.style.hide(axis="index")

Informações,Libertadores
Jogos,150
Vitórias do mandante,74
Probabilidade de vitória do mandante,49.33
Odd média do time mandante,2.36
Odd média quando o mandante venceu,1.88
Gols do mandante,213
Média de gols do mandante,1.42
Empates,37
Vitórias do visitante,39
Probabilidade de vitória do visitante,26.00


Salvando o DataFrame

In [8486]:
aba = "Dados Gerais"

salvar_df_no_excel(tabela, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Dados Gerais


## Coletivo

In [8487]:
coletivo = df.copy()

#### Mais Vitórias

In [8488]:
vencedores = (
    coletivo["vencedor"]
        .value_counts(dropna=False)
        .rename_axis("clube")
        .reset_index(name="vitorias")
        .sort_values("vitorias", ascending=False)
        .reset_index(drop=True)
)

In [8489]:
vencedores = vencedores[vencedores["clube"] != "Empate"]

In [8490]:
mais_vitorias = vencedores.head(5)

mais_vitorias

,clube,vitorias
1,Palmeiras,9
2,Racing,7
3,Estudiantes,6
4,LDU,6
5,Cerro Porteño,6


In [8491]:
aba = "Mais Vitórias"

salvar_df_no_excel(mais_vitorias, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Mais Vitórias


#### Menos Vitórias

In [8492]:
menos_vitorias = vencedores.tail(5)

menos_vitorias = menos_vitorias.sort_values("vitorias")

menos_vitorias

,clube,vitorias
35,Blooming,1
36,Sporting Cristal,1
37,Talleres,1
38,Olimpia,1
39,Colo-Colo,1


In [8493]:
aba = "Menos Vitórias"

salvar_df_no_excel(menos_vitorias, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Menos Vitórias


#### Mais derrotas

In [8494]:
perdedor = (
    coletivo["perdedor"]
        .value_counts(dropna=False)
        .rename_axis("clube")
        .reset_index(name="derrotas")
        .sort_values("derrotas", ascending=False)
        .reset_index(drop=True)
)

In [8495]:
perdedor = perdedor[perdedor["clube"] != "Empate"]

mais_derrotas = perdedor.head(5)

mais_derrotas

,clube,derrotas
1,Deportivo Táchira,6
2,Carabobo,5
3,Barcelona de Guayaquil,5
4,Cerro Porteño,4
5,Alianza Lima,4


In [8496]:
aba = "Mais Derrotas"

salvar_df_no_excel(mais_derrotas, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Mais Derrotas


#### Menos derrotas

In [8497]:
menos_derrotas = perdedor.tail(5)

menos_derrotas = menos_derrotas.sort_values("derrotas")

menos_derrotas

,clube,derrotas
41,Boca Juniors,1
42,Ñublense,1
43,Boston River,1
44,The Strongest,1
45,Corinthians,1


In [8498]:
aba = "Menos Derrotas"

salvar_df_no_excel(menos_derrotas, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Menos Derrotas


#### Média de Pontos por Jogo (CASA)

In [8499]:
pontos_por_jogo = (
    coletivo
        .groupby("casa", as_index=True)["pontos_casa"]
        .mean()
        .reset_index(name="pontuacao")
        .sort_values("pontuacao", ascending=False)
        .reset_index(drop=True)
)

pontos_por_jogo["pontuacao"] = pontos_por_jogo["pontuacao"].round(2)

In [8500]:
maior_media_ppj_casa = pontos_por_jogo.head(5)

maior_media_ppj_casa

,casa,pontuacao
0,Blooming,3.0
1,Boca Juniors,3.0
2,Botafogo,3.0
3,Santa Fe,3.0
4,Corinthians,3.0


In [8501]:
aba = "Maior Média de PPJ (CASA)"

salvar_df_no_excel(maior_media_ppj_casa, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Maior Média de PPJ (CASA)


#### Menor Média de Pontos por Jogo (CASA)

In [8502]:
menor_media_ppj = pontos_por_jogo.tail(5)

menor_media_ppj_casa = menor_media_ppj.sort_values("pontuacao")

menor_media_ppj_casa

,casa,pontuacao
43,Defensor,0.00
44,Deportivo Táchira,0.00
45,Tolima,0.00
42,Carabobo,0.33
41,Atlético Bucaramanga,0.67


In [8503]:
aba = "Menor Média de PPJ (CASA)"

salvar_df_no_excel(menor_media_ppj_casa, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Menor Média de PPJ (CASA)


#### Média de Pontos por Jogo (FORA)

In [8504]:
pontos_por_jogo = (
    coletivo
        .groupby("fora", as_index=True)["pontos_fora"]
        .mean()
        .reset_index(name="pontuacao")
        .sort_values("pontuacao", ascending=False)
        .reset_index(drop=True)
)

pontos_por_jogo["pontuacao"] = pontos_por_jogo["pontuacao"].round(2)

In [8505]:
maior_media_ppj_fora = pontos_por_jogo.head(5)

maior_media_ppj_fora

,fora,pontuacao
0,Palmeiras,3.0
1,Racing,2.0
2,Central Córdoba,2.0
3,São Paulo,2.0
4,Estudiantes,1.8


In [8506]:
aba = "Maior Média de PPJ (FORA)"

salvar_df_no_excel(maior_media_ppj_fora, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Maior Média de PPJ (FORA)


#### Menor Média de Pontos por Jogo (FORA)

In [8507]:
menor_media_ppj = pontos_por_jogo.tail(5)

menor_media_ppj_fora = menor_media_ppj.sort_values("pontuacao")

menor_media_ppj_fora

,fora,pontuacao
41,Bolívar,0.0
42,Carabobo,0.0
43,Defensor,0.0
44,Deportivo Táchira,0.0
45,Ñublense,0.0


In [8508]:
aba = "Menor Média de PPJ (FORA)"

salvar_df_no_excel(menor_media_ppj_fora, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Menor Média de PPJ (FORA)


#### Pontuação Média Geral

In [8509]:
def tabela_classificacao(
    df: pd.DataFrame,
    col_casa: str = "casa",
    col_fora: str = "fora",
    col_pts_casa: str = "pontos_casa",
    col_pts_fora: str = "pontos_fora",
    filtro_query: str | None = None,
    ordenar_por_nome_quando_empatar: bool = True
) -> pd.DataFrame:

    # 1) (Opcional) aplicar filtro por campeonato/temporada/etc.
    base = df.query(filtro_query) if filtro_query else df

    # 2) Garantir que as colunas de pontos sejam numéricas (evita problemas de tipos)
    pts_casa = pd.to_numeric(base[col_pts_casa], errors="coerce").fillna(0)
    pts_fora = pd.to_numeric(base[col_pts_fora], errors="coerce").fillna(0)

    # 3) Empilhar mandantes e visitantes em um único DF [clube, pontuacao]
    casa_df = pd.DataFrame({"clube": base[col_casa].astype(str), "pontuacao": pts_casa})
    fora_df = pd.DataFrame({"clube": base[col_fora].astype(str), "pontuacao": pts_fora})

    empilhado = pd.concat([casa_df, fora_df], ignore_index=True)

    # 4) Somar pontos por clube
    classificacao = (
        empilhado
        .groupby("clube", as_index=False)["pontuacao"]
        .mean()
    )

    # 5) Ordenar por pontuação (desc) e, opcionalmente, por nome para desempate estável
    if ordenar_por_nome_quando_empatar:
        classificacao = classificacao.sort_values(
            by=["pontuacao", "clube"], ascending=[False, True]
        )
    else:
        classificacao = classificacao.sort_values(by="pontuacao", ascending=False)

    # 6) Reindex amigável
    classificacao = classificacao.reset_index(drop=True)

    return classificacao

In [8510]:
tabela = tabela_classificacao(coletivo)

tabela["pontuacao"] = tabela["pontuacao"].round(2)

tabela.head(3)

,clube,pontuacao
0,Palmeiras,2.8
1,Racing,2.2
2,Flamengo,2.0


In [8511]:
maior_media_ppj = tabela.head(5)

maior_media_ppj

,clube,pontuacao
0,Palmeiras,2.8
1,Racing,2.2
2,Flamengo,2.0
3,LDU,2.0
4,Estudiantes,1.9


In [8512]:
aba = "Maior média de PPJ (GERAL)"

salvar_df_no_excel(maior_media_ppj, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Maior média de PPJ (GERAL)


#### Menor Média de Pontos por Jogo (GERAL)

In [8513]:
menor_media_ppj = tabela.tail(5)

menor_media_ppj = menor_media_ppj.sort_values("pontuacao")

menor_media_ppj

,clube,pontuacao
43,Defensor,0.00
44,Deportivo Táchira,0.00
45,Tolima,0.00
42,Carabobo,0.17
41,Ñublense,0.50


In [8514]:
aba = "Menor Média de PPJ (GERAL)"

salvar_df_no_excel(menor_media_ppj, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Menor Média de PPJ (GERAL)


#### Melhor Ataque

In [8515]:
def gols_marcados(
    df: pd.DataFrame,
    col_casa: str = "casa",
    col_fora: str = "fora",
    col_gols_casa: str = "gols_casa",
    col_gols_fora: str = "gols_fora",
    filtro_query: str | None = None,
    casas_decimais_media: int = 2,
    ordenar_por: str = "gols"  # "gols" ou "media"
) -> pd.DataFrame:

    base = df.query(filtro_query) if filtro_query else df

    # Garantir numérico
    gc = pd.to_numeric(base[col_gols_casa], errors="coerce").fillna(0)
    gf = pd.to_numeric(base[col_gols_fora], errors="coerce").fillna(0)

    # Long/empilhado: gols do mandante contam pro clube da casa; do visitante, pro clube de fora
    long_casa = pd.DataFrame({
        "clube": base[col_casa].astype(str),
        "gols": gc
    })
    long_fora = pd.DataFrame({
        "clube": base[col_fora].astype(str),
        "gols": gf
    })
    long_df = pd.concat([long_casa, long_fora], ignore_index=True)

    # Agrupar: soma de gols e nº de jogos
    agg = (long_df
           .groupby("clube", as_index=False)
           .agg(gols_marcados=("gols", "sum"),
                jogos=("gols", "size")))

    # Média por jogo (float)
    agg["media_gols_marcados"] = (agg["gols_marcados"] / agg["jogos"]).astype(float)
    if casas_decimais_media is not None:
        agg["media_gols_marcados"] = agg["media_gols_marcados"].round(casas_decimais_media)

    # Ordenação
    if ordenar_por == "media":
        agg = agg.sort_values(["media_gols_marcados", "clube"], ascending=[False, True])
    else:
        agg = agg.sort_values(["gols_marcados", "clube"], ascending=[False, True])

    # Só as colunas pedidas (mantendo 'clube')
    res = agg[["clube", "gols_marcados", "media_gols_marcados"]].reset_index(drop=True)
    return res

In [8516]:
gols = gols_marcados(coletivo)

gols.head(3)

,clube,gols_marcados,media_gols_marcados
0,Palmeiras,26,2.60
1,Cerro Porteño,19,1.58
2,Racing,19,1.90


In [8517]:
melhor_ataque = gols[["clube", "gols_marcados"]]

melhor_ataque = melhor_ataque.sort_values("gols_marcados", ascending=False).head(5)

melhor_ataque

,clube,gols_marcados
0,Palmeiras,26
2,Racing,19
1,Cerro Porteño,19
3,Alianza Lima,16
4,River Plate,16


In [8518]:
aba = "Melhor Ataque"

salvar_df_no_excel(melhor_ataque, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Melhor Ataque


#### Melhor Média de Gols Marcados

In [8519]:
melhor_media_gols_marcados = gols[["clube", "media_gols_marcados"]]

melhor_media_gols_marcados = melhor_media_gols_marcados.sort_values("media_gols_marcados", ascending=False).head(5)

melhor_media_gols_marcados

,clube,media_gols_marcados
0,Palmeiras,2.6
33,Blooming,2.0
8,Bolívar,2.0
2,Racing,1.9
4,River Plate,1.6


In [8520]:
aba = "Melhor Média de Gols Marcados"

salvar_df_no_excel(melhor_media_gols_marcados, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Melhor Média de Gols Marcados


#### Pior Ataque

In [8521]:
pior_ataque = gols[["clube", "gols_marcados"]]

pior_ataque = pior_ataque.sort_values("gols_marcados").head(5)

pior_ataque

,clube,gols_marcados
45,Tolima,0
44,Defensor,0
42,The Strongest,1
43,Ñublense,1
41,Carabobo,2


In [8522]:
aba = "Pior Ataque"

salvar_df_no_excel(pior_ataque, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Pior Ataque


#### Pior Média de Gols Marcados

In [8523]:
pior_media_gm = gols[["clube", "media_gols_marcados"]]

pior_media_gm = pior_media_gm.sort_values("media_gols_marcados").head(5)

pior_media_gm

,clube,media_gols_marcados
45,Tolima,0.00
44,Defensor,0.00
41,Carabobo,0.33
43,Ñublense,0.50
42,The Strongest,0.50


In [8524]:
aba = "Pior Média de GM"

salvar_df_no_excel(pior_media_gm, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Pior Média de GM


#### Melhor Defesa

In [8525]:
def gols_sofridos_funcao(
    df: pd.DataFrame,
    col_casa: str = "casa",
    col_fora: str = "fora",
    col_gols_casa: str = "gols_casa",
    col_gols_fora: str = "gols_fora",
    filtro_query: str | None = None,
    casas_decimais_media: int = 2,
    ordenar_por: str = "gols"  # "gols" (desc) ou "media" (desc)
) -> pd.DataFrame:

    base = df.query(filtro_query) if filtro_query else df

    # Garantir numérico
    gc = pd.to_numeric(base[col_gols_casa], errors="coerce").fillna(0)
    gf = pd.to_numeric(base[col_gols_fora], errors="coerce").fillna(0)

    # Long/empilhado: sofridos do mandante = gols_fora; sofridos do visitante = gols_casa
    long_mandante = pd.DataFrame({
        "clube": base[col_casa].astype(str),
        "gols_sofridos": gf
    })
    long_visitante = pd.DataFrame({
        "clube": base[col_fora].astype(str),
        "gols_sofridos": gc
    })
    long_df = pd.concat([long_mandante, long_visitante], ignore_index=True)

    # Agrupar: soma de sofridos e nº de jogos
    agg = (long_df
           .groupby("clube", as_index=False)
           .agg(gols_sofridos=("gols_sofridos", "sum"),
                jogos=("gols_sofridos", "size")))

    # Média por jogo
    agg["media_gols_sofridos"] = (agg["gols_sofridos"] / agg["jogos"]).astype(float)
    if casas_decimais_media is not None:
        agg["media_gols_sofridos"] = agg["media_gols_sofridos"].round(casas_decimais_media)

    # Ordenação
    if ordenar_por == "media":
        agg = agg.sort_values(["media_gols_sofridos", "clube"], ascending=[False, True])
    else:
        agg = agg.sort_values(["gols_sofridos", "clube"], ascending=[False, True])

    return agg[["clube", "gols_sofridos", "media_gols_sofridos"]].reset_index(drop=True)

In [8526]:
gols_sofridos = gols_sofridos_funcao(coletivo)

gols_sofridos.head(3)

,clube,gols_sofridos,media_gols_sofridos
0,Alianza Lima,17,1.42
1,Sporting Cristal,16,2.67
2,Cerro Porteño,15,1.25


In [8527]:
melhor_defesa = gols_sofridos[["clube", "gols_sofridos"]]

melhor_defesa = melhor_defesa.sort_values("gols_sofridos").head(5)

melhor_defesa

,clube,gols_sofridos
45,Ñublense,2
42,Boca Juniors,2
44,Tolima,2
43,Boston River,2
41,Santa Fe,3


In [8528]:
aba = "Melhor Defesa"

salvar_df_no_excel(melhor_defesa, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Melhor Defesa


#### Melhor Média de Gols Sofridos

In [8529]:
melhor_media_gs = gols_sofridos[["clube", "media_gols_sofridos"]]

melhor_media_gs = melhor_media_gs.sort_values("media_gols_sofridos").head(5)

melhor_media_gs

,clube,media_gols_sofridos
43,Boston River,0.5
36,Racing,0.5
34,LDU,0.5
33,Flamengo,0.5
30,Palmeiras,0.6


In [8530]:
aba = "Melhor Média de GS"

salvar_df_no_excel(melhor_media_gs, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Melhor Média de GS


#### Pior Defesa

In [8531]:
pior_defesa = gols_sofridos[["clube", "gols_sofridos"]].head(5)

pior_defesa

,clube,gols_sofridos
0,Alianza Lima,17
1,Sporting Cristal,16
2,Cerro Porteño,15
3,San Antonio Bulo-Bulo,15
4,Nacional/URU,14


In [8532]:
aba = "Pior Defesa"

salvar_df_no_excel(pior_defesa, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Pior Defesa


#### Pior Média de Gols Sofridos

In [8533]:
pior_media_gs = gols_sofridos[["clube", "media_gols_sofridos"]].head(5)

pior_media_gs

,clube,media_gols_sofridos
0,Alianza Lima,1.42
1,Sporting Cristal,2.67
2,Cerro Porteño,1.25
3,San Antonio Bulo-Bulo,2.50
4,Nacional/URU,1.75


In [8534]:
aba = "Pior Média de GS"

salvar_df_no_excel(pior_media_gs, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Pior Média de GS


#### Clean Sheets

In [8535]:
def clean_sheets_e_clean_goals(
    df: pd.DataFrame,
    col_casa: str = "casa",
    col_fora: str = "fora",
    col_gols_casa: str = "gols_casa",
    col_gols_fora: str = "gols_fora",
    filtro_query: str | None = None,
    incluir_taxas: bool = False,
    casas_decimais_taxas: int = 2,
    ordenar_por: str = "cs"
) -> pd.DataFrame:

    base = df.query(filtro_query) if filtro_query else df

    # Garantir numéricos
    gc = pd.to_numeric(base[col_gols_casa], errors="coerce").fillna(0)
    gf = pd.to_numeric(base[col_gols_fora], errors="coerce").fillna(0)

    # Linhas do mandante
    home = pd.DataFrame({
        "clube": base[col_casa].astype(str),
        "cs": (gf == 0).astype(int),
        "cg": (gc == 0).astype(int),
    })

    # Linhas do visitante
    away = pd.DataFrame({
        "clube": base[col_fora].astype(str),
        "cs": (gc == 0).astype(int),
        "cg": (gf == 0).astype(int),
    })

    long_df = pd.concat([home, away], ignore_index=True)

    # Agregar por clube
    agg = (long_df
           .groupby("clube", as_index=False)
           .agg(clean_sheets=("cs", "sum"),
                clean_goals=("cg", "sum"),
                jogos=("cs", "size")))

    if incluir_taxas:
        agg["tx_clean_sheets"] = (agg["clean_sheets"] / agg["jogos"]).round(casas_decimais_taxas)
        agg["tx_clean_goals"]  = (agg["clean_goals"]  / agg["jogos"]).round(casas_decimais_taxas)

    # Ordenação
    if   ordenar_por == "jogos":
        agg = agg.sort_values(["jogos", "clube"], ascending=[False, True])
    elif ordenar_por == "cg":
        agg = agg.sort_values(["clean_goals", "clube"], ascending=[False, True])
    else:  # "cs"
        agg = agg.sort_values(["clean_sheets", "clube"], ascending=[False, True])

    cols = ["clube", "clean_sheets", "clean_goals"]
    if incluir_taxas:
        cols += ["tx_clean_sheets", "tx_clean_goals", "jogos"]

    return agg[cols].reset_index(drop=True)

In [8536]:
jogos_clean = clean_sheets_e_clean_goals(coletivo)

jogos_clean.head(3)

,clube,clean_sheets,clean_goals
0,Estudiantes,7,1
1,LDU,7,4
2,Flamengo,6,2


In [8537]:
clean_sheets = jogos_clean[["clube", "clean_sheets"]]

mais_clean_sheets = clean_sheets.head(5)

mais_clean_sheets

,clube,clean_sheets
0,Estudiantes,7
1,LDU,7
2,Flamengo,6
3,Palmeiras,6
4,Racing,6


In [8538]:
aba = "Mais Clean Sheets"

salvar_df_no_excel(mais_clean_sheets, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Mais Clean Sheets


#### Menos Clean Sheets

In [8539]:
menos_clean_sheets = clean_sheets.sort_values("clean_sheets").head(5)

menos_clean_sheets

,clube,clean_sheets
45,Ñublense,0
32,Blooming,0
34,Carabobo,0
35,Defensor,0
36,Deportes Iquique,0


In [8540]:
aba = "Menos Clean Sheets"

salvar_df_no_excel(menos_clean_sheets, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Menos Clean Sheets


#### Menos Clean Goals

In [8541]:
clean_goals = jogos_clean[["clube", "clean_goals"]]

menos_clean_goals = clean_goals.sort_values("clean_goals").head(5)

menos_clean_goals

,clube,clean_goals
40,Santa Fe,0
36,Deportes Iquique,0
32,Blooming,0
44,UCV,0
0,Estudiantes,1


In [8542]:
aba = "Menos Clean Goals"

salvar_df_no_excel(menos_clean_goals, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Menos Clean Goals


#### Mais Clean Goals

In [8543]:
mais_clean_goals = clean_goals.sort_values("clean_goals", ascending=False).head(5)

mais_clean_goals

,clube,clean_goals
10,Fortaleza,6
9,Cerro Porteño,5
16,Universitario,4
8,Barcelona de Guayaquil,4
1,LDU,4


In [8544]:
aba = "Mais Clean Goals"

salvar_df_no_excel(mais_clean_goals, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Mais Clean Goals


#### Invencibilidade

In [8545]:
def invencibilidade(
    df: pd.DataFrame,
    col_casa: str = "casa",
    col_fora: str = "fora",
    col_gols_casa: str = "gols_casa",
    col_gols_fora: str = "gols_fora",
    col_data: str | None = None,
    filtro_query: str | None = None,
    ordenar_por: str = "atual"
) -> pd.DataFrame:

    base = df.query(filtro_query) if filtro_query else df

    # Garantir numéricos
    gc = pd.to_numeric(base[col_gols_casa], errors="coerce").fillna(0)
    gf = pd.to_numeric(base[col_gols_fora], errors="coerce").fillna(0)

    # Coluna de ordenação
    if col_data and col_data in base.columns:
        # Tenta converter para datetime; se não der, usa como está
        try:
            ordem = pd.to_datetime(base[col_data], errors="coerce")
            # Fallback: se tudo virou NaT, use a própria coluna como está
            if ordem.isna().all():
                ordem = base[col_data]
        except Exception:
            ordem = base[col_data]
    else:
        # Usa o índice atual como ordem (pressupõe já ordenado)
        ordem = base.index

    # Monta visão "long" com uma linha por clube por jogo e flag "nao_perdeu"
    home = pd.DataFrame({
        "clube": base[col_casa].astype(str),
        "ordem": ordem,
        "nao_perdeu": (gc >= gf).astype(bool)
    })
    away = pd.DataFrame({
        "clube": base[col_fora].astype(str),
        "ordem": ordem,
        "nao_perdeu": (gf >= gc).astype(bool)
    })
    long_df = pd.concat([home, away], ignore_index=True)

    # Ordena cronologicamente dentro de cada clube
    long_df = long_df.sort_values(["clube", "ordem"], kind="mergesort")  # mergesort mantém estabilidade

    # Funções auxiliares para sequência
    def _streaks(serie_bool: pd.Series) -> tuple[int, int]:

        # Maior sequência
        best = cur = 0
        for v in serie_bool:
            if v:
                cur += 1
                if cur > best:
                    best = cur
            else:
                cur = 0

        # Sequência atual (do fim pra trás)
        cur_end = 0
        for v in reversed(serie_bool.tolist()):
            if v:
                cur_end += 1
            else:
                break

        return best, cur_end

    # Calcula por clube
    resultados = []
    for clube, g in long_df.groupby("clube", sort=False):
        nao_perdeu_seq = g["nao_perdeu"]
        max_streak, current_streak = _streaks(nao_perdeu_seq)
        resultados.append({
            "clube": clube,
            "maior_invencibilidade": int(max_streak),
            "invencibilidade_atual": int(current_streak),
        })

    out = pd.DataFrame(resultados)

    # Ordenação da saída
    if ordenar_por == "max":
        out = out.sort_values(["maior_invencibilidade", "clube"], ascending=[False, True])
    else:  # "atual"
        out = out.sort_values(["invencibilidade_atual", "clube"], ascending=[False, True])

    return out.reset_index(drop=True)

In [8546]:
jogos_invictos = invencibilidade(coletivo)

jogos_invictos.head(5)

,clube,maior_invencibilidade,invencibilidade_atual
0,Palmeiras,10,10
1,LDU,4,3
2,Racing,4,3
3,Atlético Nacional,2,2
4,Libertad,5,2


In [8547]:
invencibilidade_historica = jogos_invictos[["clube", "maior_invencibilidade"]]

maior_invencibilidade = invencibilidade_historica.sort_values("maior_invencibilidade", ascending=False).head(5)

maior_invencibilidade

,clube,maior_invencibilidade
0,Palmeiras,10
39,São Paulo,8
36,River Plate,8
30,Flamengo,7
21,Bahia,7


In [8548]:
aba = "Maior Invencibilidade"

salvar_df_no_excel(maior_invencibilidade, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Maior Invencibilidade


#### Menor Invencibilidade

In [8549]:
menor_invencibilidade = invencibilidade_historica.sort_values("maior_invencibilidade").head(5)

menor_invencibilidade

,clube,maior_invencibilidade
29,Deportivo Táchira,0
28,Defensor,0
42,Tolima,0
17,Santa Fe,1
26,Carabobo,1


In [8550]:
aba = "Menor Invencibilidade"

salvar_df_no_excel(menor_invencibilidade, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Menor Invencibilidade


#### Invencilidade Atual

In [8551]:
invencibilidade_atual = jogos_invictos[["clube", "invencibilidade_atual"]].head(5)

invencibilidade_atual

,clube,invencibilidade_atual
0,Palmeiras,10
1,LDU,3
2,Racing,3
3,Atlético Nacional,2
4,Libertad,2


In [8552]:
aba = "Invencibilidade Atual"

salvar_df_no_excel(invencibilidade_atual, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Invencibilidade Atual


#### Sequência de Vitórias

In [8553]:
def vitorias_em_sequencia(
    df: pd.DataFrame,
    col_casa: str = "casa",
    col_fora: str = "fora",
    col_gols_casa: str = "gols_casa",
    col_gols_fora: str = "gols_fora",
    col_data: str | None = None,
    filtro_query: str | None = None,
    ordenar_por: str = "atual"
) -> pd.DataFrame:

    base = df.query(filtro_query) if filtro_query else df

    # Garantir numéricos
    gc = pd.to_numeric(base[col_gols_casa], errors="coerce").fillna(0)
    gf = pd.to_numeric(base[col_gols_fora], errors="coerce").fillna(0)

    # Coluna de ordenação cronológica
    if col_data and col_data in base.columns:
        try:
            ordem = pd.to_datetime(base[col_data], errors="coerce")
            if ordem.isna().all():
                ordem = base[col_data]
        except Exception:
            ordem = base[col_data]
    else:
        ordem = base.index  # usa ordem do DF

    # Visão "long": uma linha por clube por jogo, com flag de vitória
    home = pd.DataFrame({
        "clube": base[col_casa].astype(str),
        "ordem": ordem,
        "venceu": (gc > gf).astype(bool),
    })
    away = pd.DataFrame({
        "clube": base[col_fora].astype(str),
        "ordem": ordem,
        "venceu": (gf > gc).astype(bool),
    })
    long_df = pd.concat([home, away], ignore_index=True)

    # Ordena cronologicamente dentro de cada clube
    long_df = long_df.sort_values(["clube", "ordem"], kind="mergesort")

    # Função de cálculo das sequências
    def _streaks(serie_bool: pd.Series) -> tuple[int, int]:
        best = cur = 0
        for v in serie_bool:
            if v:
                cur += 1
                if cur > best:
                    best = cur
            else:
                cur = 0
        # Sequência atual (do fim para o início)
        cur_end = 0
        for v in reversed(serie_bool.tolist()):
            if v:
                cur_end += 1
            else:
                break
        return best, cur_end

    resultados = []
    for clube, g in long_df.groupby("clube", sort=False):
        venceu_seq = g["venceu"]
        max_streak, current_streak = _streaks(venceu_seq)
        resultados.append({
            "clube": clube,
            "maximo_vitorias": int(max_streak),
            "sequencia_vitorias_atual": int(current_streak),
        })

    out = pd.DataFrame(resultados)

    # Ordenação final
    if ordenar_por == "max":
        out = out.sort_values(["maximo_vitorias", "clube"], ascending=[False, True])
    else:
        out = out.sort_values(["sequencia_vitorias_atual", "clube"], ascending=[False, True])

    return out.reset_index(drop=True)

In [8554]:
vitorias_seguidas = vitorias_em_sequencia(coletivo)

vitorias_seguidas.head()

,clube,maximo_vitorias,sequencia_vitorias_atual
0,LDU,3,3
1,Racing,3,3
2,Palmeiras,7,2
3,Boca Juniors,1,1
4,Bolívar,1,1


In [8555]:
vitorias_historico = vitorias_seguidas[["clube", "maximo_vitorias"]]

maximo_historico = vitorias_historico.sort_values("maximo_vitorias", ascending=False).head(5)

maximo_historico

,clube,maximo_vitorias
2,Palmeiras,7
27,Flamengo,5
22,Cerro Porteño,5
19,Botafogo,4
0,LDU,3


In [8556]:
aba = "Maior Sequência de Vitórias"

salvar_df_no_excel(maximo_historico, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Maior Sequência de Vitórias


#### Menor Sequência de Vitórias

In [8557]:
minimo_historico = vitorias_historico.sort_values("maximo_vitorias", ascending=True).head(5)

minimo_historico

,clube,maximo_vitorias
45,Ñublense,0
40,Tolima,0
25,Deportivo Táchira,0
23,Defensor,0
20,Carabobo,0


In [8558]:
aba = "Menor Sequência de Vitórias"

salvar_df_no_excel(minimo_historico, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Menor Sequência de Vitórias


#### Maior Sequência Atual de Vitórias

In [8559]:
sequencia_vitorias_atual = vitorias_seguidas[["clube", "sequencia_vitorias_atual"]].head(5)

sequencia_vitorias_atual

,clube,sequencia_vitorias_atual
0,LDU,3
1,Racing,3
2,Palmeiras,2
3,Boca Juniors,1
4,Bolívar,1


In [8560]:
aba = "Sequência Atual de Vitórias"

salvar_df_no_excel(sequencia_vitorias_atual, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Sequência Atual de Vitórias


#### Carregando a consulta

In [8561]:
conn = mysql.connector.connect(**config)

In [8562]:
query = f"""
SELECT
  p.data,
    ep.clube,
    ep.ball_possession,
    ep.expected_goals,
    ep.total_shots,
    ep.shots_on_target,
    ep.shots_inside_box,
    ep.final_third_entries,
    ep.big_chances,
    ep.big_chances_scored,
    ep.goalkeeper_saves,
    ep.passes,
    ep.accurate_passes,
    ep.duels
FROM partidas p
JOIN estatisticas_partidas ep ON p.id_jogo = ep.id_jogo
WHERE p.temporada = '{temporada}'
	AND p.campeonato = '{campeonato}'
	AND p.pais = '{pais}';
"""

In [8563]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_30808\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [8564]:
conn.close()

#### Posse de bola média

In [8565]:
posse_bola_media = df[["clube", "ball_possession"]]

In [8566]:
posse_bola_media['ball_possession'] = posse_bola_media['ball_possession'].str.replace('%', '', regex=False).astype(float)

posse_bola_media['ball_possession'] = (posse_bola_media['ball_possession'] / 100) * 100

posse_bola_media.head(5)

C:\Users\stefa\AppData\Local\Temp\ipykernel_30808\42955439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posse_bola_media['ball_possession'] = posse_bola_media['ball_possession'].str.replace('%', '', regex=False).astype(float)
C:\Users\stefa\AppData\Local\Temp\ipykernel_30808\42955439.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posse_bola_media['ball_possession'] = (posse_bola_media['ball_possession'] / 100) * 100


,clube,ball_possession
0,Monagas,39.0
1,Defensor,61.0
2,Nacional/URU,30.0
3,Alianza Lima,70.0
4,Blooming,37.0


In [8567]:
posse_bola_media = (
    posse_bola_media
        .groupby("clube", as_index=True)["ball_possession"]
        .mean()
        .reset_index(name="posse_bola_media")
        .sort_values("posse_bola_media", ascending=False)
        .reset_index(drop=True)
)

In [8568]:
posse_bola_media = posse_bola_media.round(2).head(10)

posse_bola_media

,clube,posse_bola_media
0,Boca Juniors,70.50
1,Bolívar,67.67
2,Tolima,64.50
3,Corinthians,64.00
4,El Nacional,64.00
5,Flamengo,61.70
6,Bahia,61.60
7,Santa Fe,61.50
8,River Plate,59.90
9,Defensor,59.50


In [8569]:
aba = "Posse de Bola Média"

salvar_df_no_excel(posse_bola_media, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Posse de Bola Média


#### Maior xG Médio

In [8570]:
xg_medio = df[["clube", "expected_goals"]]

xg_medio.head(3)

,clube,expected_goals
0,Monagas,1.08
1,Defensor,0.46
2,Nacional/URU,1.10


In [8571]:
xg_medio = (
    xg_medio
        .groupby("clube", as_index=True)["expected_goals"]
        .mean()
        .reset_index(name="xg_medio")
        .sort_values("xg_medio", ascending=False)
        .reset_index(drop=True)
)

In [8572]:
xg_medio = xg_medio.round(2)

maior_xg_medio = xg_medio.head(10)

maior_xg_medio

,clube,xg_medio
0,Palmeiras,2.07
1,Racing,1.72
2,River Plate,1.65
3,Bolívar,1.64
4,Santa Fe,1.55
5,Cerro Porteño,1.51
6,Universidad de Chile,1.46
7,Bahia,1.46
8,Central Córdoba,1.46
9,Peñarol,1.41


In [8573]:
aba = "Maior xG Médio"

salvar_df_no_excel(maior_xg_medio, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Maior xG Médio


#### Menor xG Médio

In [8574]:
menor_xg_medio = xg_medio.sort_values("xg_medio").head(10)

menor_xg_medio

,clube,xg_medio
45,Carabobo,0.43
44,Monagas,0.54
43,San Antonio Bulo-Bulo,0.64
42,Defensor,0.65
41,Ñublense,0.74
40,The Strongest,0.78
39,Sporting Cristal,0.79
38,Universitario,0.81
37,Talleres,0.82
36,Atlético Bucaramanga,0.88


In [8575]:
aba = "Menor xG Médio"

salvar_df_no_excel(menor_xg_medio, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Menor xG Médio


#### Média de Finalizações

In [8576]:
total_finalizacoes = df[["clube", "total_shots"]]

total_finalizacoes.head(3)

,clube,total_shots
0,Monagas,19
1,Defensor,14
2,Nacional/URU,11


In [8577]:
media_finalizacoes = (
    total_finalizacoes
        .groupby("clube", as_index=True)["total_shots"]
        .mean()
        .reset_index(name="media_finalizacoes")
        .sort_values("media_finalizacoes", ascending=False)
        .reset_index(drop=True)
)

In [8578]:
media_finalizacoes["media_finalizacoes"] = media_finalizacoes["media_finalizacoes"].astype(int)

maior_media_finalizacoes = media_finalizacoes.head(10)

maior_media_finalizacoes

,clube,media_finalizacoes
0,Santa Fe,22
1,Olimpia,19
2,Corinthians,17
3,Boca Juniors,17
4,Bolívar,17
5,Racing,16
6,River Plate,16
7,Defensor,16
8,Talleres,15
9,Bahia,15


In [8579]:
aba = "Maior Média de Finalizações"

salvar_df_no_excel(maior_media_finalizacoes, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Maior Média de Finalizações


#### Maior Média de Finalizações Certas

In [8580]:
finalizacoes_certas = df[["clube", "shots_on_target"]]

finalizacoes_certas.head(3)

,clube,shots_on_target
0,Monagas,9
1,Defensor,3
2,Nacional/URU,3


In [8581]:
media_finalizacoes_certas = (
    finalizacoes_certas
        .groupby("clube", as_index=True)["shots_on_target"]
        .mean()
        .reset_index(name="finalizacoes_certas")
        .sort_values("finalizacoes_certas", ascending=False)
        .reset_index(drop=True)
)

In [8582]:
media_finalizacoes_certas["finalizacoes_certas"] = media_finalizacoes_certas["finalizacoes_certas"].astype(int)

maior_media_finalizacoes_certas = media_finalizacoes_certas.head(10)

maior_media_finalizacoes_certas

,clube,finalizacoes_certas
0,Santa Fe,9
1,Olimpia,7
2,Palmeiras,7
3,Boca Juniors,7
4,UCV,6
5,Racing,6
6,River Plate,5
7,Bolívar,5
8,Central Córdoba,5
9,Peñarol,5


In [8583]:
aba = "Maior Média de Fin. Certas"

salvar_df_no_excel(maior_media_finalizacoes_certas, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Maior Média de Fin. Certas


#### Aproveitamento das Finalizações

In [8584]:
aproveitamento_finalizacoes = df[["clube", "total_shots", "shots_on_target"]]

aproveitamento_finalizacoes.head(3)

,clube,total_shots,shots_on_target
0,Monagas,19,9
1,Defensor,14,3
2,Nacional/URU,11,3


In [8585]:
aproveitamento_finalizacoes["aproveitamento"] = round((aproveitamento_finalizacoes["shots_on_target"] / aproveitamento_finalizacoes["total_shots"]) * 100, 2)

aproveitamento_finalizacoes.head(3)

C:\Users\stefa\AppData\Local\Temp\ipykernel_30808\426629620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aproveitamento_finalizacoes["aproveitamento"] = round((aproveitamento_finalizacoes["shots_on_target"] / aproveitamento_finalizacoes["total_shots"]) * 100, 2)


,clube,total_shots,shots_on_target,aproveitamento
0,Monagas,19,9,47.37
1,Defensor,14,3,21.43
2,Nacional/URU,11,3,27.27


In [8586]:
aproveitamento_finalizacoes = aproveitamento_finalizacoes[["clube", "aproveitamento"]]

aproveitamento_finalizacoes.head(3)

,clube,aproveitamento
0,Monagas,47.37
1,Defensor,21.43
2,Nacional/URU,27.27


In [8587]:
aproveitamento_finalizacoes = (
    aproveitamento_finalizacoes
        .groupby("clube", as_index=False)["aproveitamento"]
        .mean()
        .sort_values("aproveitamento", ascending=False)
        .reset_index(drop=True)
)

In [8588]:
aproveitamento_finalizacoes["aproveitamento"] = aproveitamento_finalizacoes["aproveitamento"].round(2)

aproveitamento_finalizacoes = aproveitamento_finalizacoes.head(10)

aproveitamento_finalizacoes.head(3)

,clube,aproveitamento
0,UCV,60.58
1,Palmeiras,47.26
2,Blooming,46.42


In [8589]:
aba = "Aproveitamento das Finalizações"

salvar_df_no_excel(aproveitamento_finalizacoes, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Aproveitamento das Finalizações


#### Finalizações de Dentro da Área

In [8590]:
finalizacao_dentro_area = df.copy()

finalizacao_dentro_area = finalizacao_dentro_area[["clube", "shots_inside_box"]]

finalizacao_dentro_area.head(3)

,clube,shots_inside_box
0,Monagas,10
1,Defensor,5
2,Nacional/URU,6


In [8591]:
finalizacao_dentro_area = (
    finalizacao_dentro_area
        .groupby("clube", as_index=True)["shots_inside_box"]
        .mean()
        .reset_index(name="media_finalizacao_dentro_area")
        .sort_values("media_finalizacao_dentro_area", ascending=False)
        .reset_index(drop=True)
)

In [8592]:
finalizacao_dentro_area["media_finalizacao_dentro_area"] = finalizacao_dentro_area["media_finalizacao_dentro_area"].astype(int)

media_finalizacao_dentro_area = finalizacao_dentro_area.head(10)

media_finalizacao_dentro_area

,clube,media_finalizacao_dentro_area
0,Santa Fe,13
1,Racing,11
2,Corinthians,10
3,Bahia,9
4,Olimpia,9
5,Palmeiras,9
6,Tolima,9
7,Bolívar,9
8,River Plate,9
9,Boca Juniors,9


In [8593]:
aba = "Média de Fin. Dentro Área"

salvar_df_no_excel(media_finalizacao_dentro_area, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Média de Fin. Dentro Área


#### Entradas no Terço final

In [8594]:
entradas_terco_final = df.copy()

In [8595]:
entradas_terco_final = entradas_terco_final[["clube", "final_third_entries"]]

entradas_terco_final.head(3)

,clube,final_third_entries
0,Monagas,51
1,Defensor,81
2,Nacional/URU,60


In [8596]:
entradas_terco_final = (
    entradas_terco_final
        .groupby("clube", as_index=True)["final_third_entries"]
        .mean()
        .reset_index(name="entradas_terco_final")
        .sort_values("entradas_terco_final", ascending=False)
        .reset_index(drop=True)
)

In [8597]:
entradas_terco_final["entradas_terco_final"] = entradas_terco_final["entradas_terco_final"].astype(int)

entradas_terco_final = entradas_terco_final.head(10)

entradas_terco_final

,clube,entradas_terco_final
0,Defensor,85
1,Boca Juniors,83
2,Tolima,73
3,El Nacional,72
4,Santa Fe,65
5,Flamengo,64
6,Racing,64
7,River Plate,64
8,Melgar,63
9,Olimpia,62


In [8598]:
aba = "Entradas no Terço Final"

salvar_df_no_excel(entradas_terco_final, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Entradas no Terço Final


#### Grandes Chances Criadas

In [8599]:
grandes_chances = df.copy()

grandes_chances = grandes_chances[["clube", "big_chances"]]

grandes_chances.head(3)

,clube,big_chances
0,Monagas,3
1,Defensor,0
2,Nacional/URU,1


In [8600]:
grandes_chances = (
    grandes_chances
        .groupby("clube", as_index=True)["big_chances"]
        .mean()
        .reset_index(name="grandes_chances")
        .sort_values("grandes_chances", ascending=False)
        .reset_index(drop=True)
)

In [8601]:
grandes_chances["grandes_chances"] = grandes_chances["grandes_chances"].round(2)

grandes_chances = grandes_chances.head(10)

grandes_chances

,clube,grandes_chances
0,Bolívar,3.50
1,Palmeiras,3.30
2,Racing,3.10
3,Deportes Iquique,3.00
4,El Nacional,2.75
5,Central Córdoba,2.67
6,Flamengo,2.60
7,Internacional,2.50
8,River Plate,2.40
9,Peñarol,2.38


In [8602]:
aba = "Grandes Chances Clubes"

salvar_df_no_excel(grandes_chances, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Grandes Chances Clubes


#### Aproveitamento das Grandes Chances

In [8603]:
aproveitamento_grandes_chances = df[["clube", "big_chances", "big_chances_scored"]]

aproveitamento_grandes_chances.head(3)

,clube,big_chances,big_chances_scored
0,Monagas,3,1
1,Defensor,0,0
2,Nacional/URU,1,1


In [8604]:
aproveitamento_grandes_chances["aproveitamento"] = round((aproveitamento_grandes_chances["big_chances_scored"] / aproveitamento_grandes_chances["big_chances"]) * 100, 2)

aproveitamento_grandes_chances.head(3)

C:\Users\stefa\AppData\Local\Temp\ipykernel_30808\3457242034.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aproveitamento_grandes_chances["aproveitamento"] = round((aproveitamento_grandes_chances["big_chances_scored"] / aproveitamento_grandes_chances["big_chances"]) * 100, 2)


,clube,big_chances,big_chances_scored,aproveitamento
0,Monagas,3,1,33.33
1,Defensor,0,0,NaN
2,Nacional/URU,1,1,100.00


In [8605]:
aproveitamento_grandes_chances = aproveitamento_grandes_chances[["clube", "aproveitamento"]]

In [8606]:
aproveitamento_grandes_chances = (
    aproveitamento_grandes_chances
        .groupby("clube", as_index=True)["aproveitamento"]
        .mean()
        .reset_index(name="media_aprov_grandes_chances")
        .sort_values("media_aprov_grandes_chances", ascending=False)
        .reset_index(drop=True)
)

In [8607]:
aproveitamento_grandes_chances["media_aprov_grandes_chances"] = aproveitamento_grandes_chances["media_aprov_grandes_chances"].round(2)

aproveitamento_grandes_chances = aproveitamento_grandes_chances.head(10)

aproveitamento_grandes_chances

,clube,media_aprov_grandes_chances
0,Palmeiras,68.52
1,Carabobo,66.67
2,Melgar,66.66
3,Vélez Sarsfield,64.29
4,Universidad de Chile,56.67
5,Atlético Bucaramanga,54.17
6,Internacional,52.62
7,UCV,50.00
8,Santa Fe,50.00
9,San Antonio Bulo-Bulo,50.00


In [8608]:
aba = "Aproveitamento Grandes Chances"

salvar_df_no_excel(aproveitamento_grandes_chances, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Aproveitamento Grandes Chances


#### Defesas do Goleiro

In [8609]:
defesas_goleiro = df.copy()

defesas_goleiro = defesas_goleiro[["clube", "goalkeeper_saves"]]

defesas_goleiro.head(3)

,clube,goalkeeper_saves
0,Monagas,3
1,Defensor,7
2,Nacional/URU,3


In [8610]:
defesas_goleiro = (
    defesas_goleiro
        .groupby("clube", as_index=True)["goalkeeper_saves"]
        .mean()
        .reset_index(name="defesas_goleiro")
        .sort_values("defesas_goleiro", ascending=False)
        .reset_index(drop=True)
)

In [8611]:
defesas_goleiro["defesas_goleiro"] = defesas_goleiro["defesas_goleiro"].round(2)

defesas_goleiro = defesas_goleiro.head(10)

defesas_goleiro

,clube,defesas_goleiro
0,The Strongest,7.00
1,San Antonio Bulo-Bulo,5.17
2,Deportes Iquique,5.00
3,Sporting Cristal,4.67
4,Monagas,4.50
5,Carabobo,4.33
6,Deportivo Táchira,4.33
7,LDU,4.20
8,UCV,4.00
9,Atlético Bucaramanga,4.00


In [8612]:
aba = "Defesas do Goleiro"

salvar_df_no_excel(defesas_goleiro, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Defesas do Goleiro


#### Aproveitamento dos Passes

In [8613]:
aproveitamento_passes = df[["clube", "passes", "accurate_passes"]]

aproveitamento_passes.head(3)

,clube,passes,accurate_passes
0,Monagas,264,169
1,Defensor,415,306
2,Nacional/URU,185,101


In [8614]:
aproveitamento_passes["aproveitamento"] = round((aproveitamento_passes["accurate_passes"] / aproveitamento_passes["passes"]) * 100, 2)

aproveitamento_passes.head(3)

C:\Users\stefa\AppData\Local\Temp\ipykernel_30808\3494593267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aproveitamento_passes["aproveitamento"] = round((aproveitamento_passes["accurate_passes"] / aproveitamento_passes["passes"]) * 100, 2)


,clube,passes,accurate_passes,aproveitamento
0,Monagas,264,169,64.02
1,Defensor,415,306,73.73
2,Nacional/URU,185,101,54.59


In [8615]:
aproveitamento_passes = aproveitamento_passes[["clube", "aproveitamento"]]

In [8616]:
aproveitamento_passes = (
    aproveitamento_passes
        .groupby("clube")["aproveitamento"]
        .mean()
        .reset_index(name="aproveitamento_passes")
        .sort_values("aproveitamento_passes", ascending=False)
        .reset_index(drop=True)
)

In [8617]:
aproveitamento_passes["aproveitamento_passes"] = aproveitamento_passes["aproveitamento_passes"].round(2)

aproveitamento_passes = aproveitamento_passes.head(10)

aproveitamento_passes

,clube,aproveitamento_passes
0,Bahia,88.04
1,Boca Juniors,87.22
2,Bolívar,86.79
3,Flamengo,86.33
4,Corinthians,85.85
5,São Paulo,85.50
6,Atlético Nacional,84.42
7,El Nacional,84.26
8,Deportivo Táchira,84.18
9,Independiente Del Valle,83.91


In [8618]:
aba = "Aproveitamento nos Passes"

salvar_df_no_excel(aproveitamento_passes, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Aproveitamento nos Passes


## Jogadores

### Fazendo a conexão e a consulta no banco de dados

In [8619]:
conn = mysql.connector.connect(**config)

In [8620]:
query = f"""
SELECT
  p.data,
    p.id_jogo,
    ej.temporada,
    CASE
        WHEN ej.team = p.casa THEN 'Casa'
        WHEN ej.team = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    ej.player_id,
    ej.name,
    CASE
        WHEN ej.position = 'G' THEN 'Goleiro'
        WHEN ej.position = 'D' THEN 'Defensor'
        WHEN ej.position = 'M' THEN 'Meio-campo'
        WHEN ej.position = 'F' THEN 'Atacante'
        ELSE 'Verificar'
    END AS position,
    ej.team,
    ej.rating,
    ej.minutes_played,
    ej.goals,
    ej.goal_assist,
    ej.big_chance_created,
    ej.key_pass,
    ej.expected_goals
FROM partidas p
JOIN estatisticas_jogadores ej ON p.id_jogo = ej.id_jogo
WHERE p.temporada = '{temporada}'
	AND ej.minutes_played > 0
	AND p.campeonato = '{campeonato}'
	AND p.pais = '{pais}';
"""

In [8621]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_30808\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [8622]:
conn.close()

### Mais Minutos

Criando uma cópia do DataFrame original

In [8623]:
mais_minutos = df.copy()

Agrupando os jogadores e somando os minutos

In [8624]:
mais_minutos = (
    mais_minutos
        .groupby(["player_id", "name", "position", "team"], as_index=False)["minutes_played"]
        .sum()
        .rename(columns={"minutes_played": "minutos"})
        .sort_values("minutos", ascending=False)
        . reset_index(drop=True)
)

In [8625]:
mais_minutos.head(3)

,player_id,name,position,team,minutos
0,331437,Guillermo Viscarra,Goleiro,Alianza Lima,1080
1,338957,Eryc Castillo,Atacante,Alianza Lima,1073
2,1020375,Erick Noriega,Defensor,Alianza Lima,990


Montando o top 10 por posição

Goleiros

In [8626]:
mais_minutos_goleiros = mais_minutos[mais_minutos["position"] == "Goleiro"].head(10)

mais_minutos_goleiros.head(3)

,player_id,name,position,team,minutos
0,331437,Guillermo Viscarra,Goleiro,Alianza Lima,1080
3,898750,Tomás Marchiori,Goleiro,Vélez Sarsfield,900
4,1140986,Agustín Rossi,Goleiro,Flamengo,900


Eliminado a primeira coluna

In [8627]:
mais_minutos_goleiros = eliminar_primeira_coluna(mais_minutos_goleiros)

mais_minutos_goleiros.head(3)

,name,position,team,minutos
0,Guillermo Viscarra,Goleiro,Alianza Lima,1080
3,Tomás Marchiori,Goleiro,Vélez Sarsfield,900
4,Agustín Rossi,Goleiro,Flamengo,900


Salvando a informação

In [8628]:
aba = "Minutos Goleiros"

salvar_df_no_excel(mais_minutos_goleiros, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Minutos Goleiros


Defensores

In [8629]:
mais_minutos_defensores = mais_minutos[mais_minutos["position"] == "Defensor"].head(10)

mais_minutos_defensores.head(3)

,player_id,name,position,team,minutos
2,1020375,Erick Noriega,Defensor,Alianza Lima,990
5,877584,Gustavo Vallecilla,Defensor,Barcelona de Guayaquil,900
6,922853,Leonel Quiñónez,Defensor,LDU,900


Excluindo a primeira coluna

In [8630]:
mais_minutos_defensores = eliminar_primeira_coluna(mais_minutos_defensores)

mais_minutos_defensores.head(3)

,name,position,team,minutos
2,Erick Noriega,Defensor,Alianza Lima,990
5,Gustavo Vallecilla,Defensor,Barcelona de Guayaquil,900
6,Leonel Quiñónez,Defensor,LDU,900


Salvando o arquivo

In [8631]:
aba = "Minutos Defensores"

salvar_df_no_excel(mais_minutos_defensores, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Minutos Defensores


Meio-campo

In [8632]:
mais_minutos_meias = mais_minutos[mais_minutos["position"] == "Meio-campo"].head(10)

mais_minutos_meias.head(3)

,player_id,name,position,team,minutos
16,868275,Janner Corozo,Meio-campo,Barcelona de Guayaquil,869
18,830667,Santiago Ascacíbar,Meio-campo,Estudiantes,851
24,927975,Jean Lucas,Meio-campo,Bahia,840


Eliminando a primeira coluna

In [8633]:
mais_minutos_meias = eliminar_primeira_coluna(mais_minutos_meias)

mais_minutos_meias.head(3)

,name,position,team,minutos
16,Janner Corozo,Meio-campo,Barcelona de Guayaquil,869
18,Santiago Ascacíbar,Meio-campo,Estudiantes,851
24,Jean Lucas,Meio-campo,Bahia,840


Salvando o arquivo

In [8634]:
aba = "Minutos Meias"

salvar_df_no_excel(mais_minutos_meias, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Minutos Meias


Atacantes

In [8635]:
mais_minutos_atacantes = mais_minutos[mais_minutos["position"] == "Atacante"].head(10)

mais_minutos_atacantes.head(3)

,player_id,name,position,team,minutos
1,338957,Eryc Castillo,Atacante,Alianza Lima,1073
11,876929,Kevin Quevedo,Atacante,Alianza Lima,891
15,906811,Adrián Martínez,Atacante,Racing,869


Eliminando a primeira coluna

In [8636]:
mais_minutos_atacantes = eliminar_primeira_coluna(mais_minutos_atacantes)

mais_minutos_atacantes.head(3)

,name,position,team,minutos
1,Eryc Castillo,Atacante,Alianza Lima,1073
11,Kevin Quevedo,Atacante,Alianza Lima,891
15,Adrián Martínez,Atacante,Racing,869


Salvando o arquivo

In [8637]:
aba = "Minutos Atacantes"

salvar_df_no_excel(mais_minutos_atacantes, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Minutos Atacantes


### Artilharia

In [8638]:
gols = df.copy()

In [8639]:
gols = (
    gols
        .groupby(["player_id", "name", "position", "team"], as_index=False)["goals"]
        .sum()
        .rename(columns={"goals": "gols"})
        .sort_values("gols", ascending=False)
        .reset_index(drop=True)
)

Eliminando quem não fez gol

In [8640]:
gols = gols[gols["gols"] > 0]

Selecionando apenas o top 10

In [8641]:
gols = gols.head(10)

gols.head(3)

,player_id,name,position,team,gols
0,906811,Adrián Martínez,Atacante,Racing,7
1,1094179,José López,Atacante,Palmeiras,7
2,1597274,Maher Carrizo,Atacante,Vélez Sarsfield,5


Eliminando a primeira coluna

In [8642]:
gols = eliminar_primeira_coluna(gols)

gols.head(3)

,name,position,team,gols
0,Adrián Martínez,Atacante,Racing,7
1,José López,Atacante,Palmeiras,7
2,Maher Carrizo,Atacante,Vélez Sarsfield,5


Salvando o arquivo

In [8643]:
aba = "Artilharia"

salvar_df_no_excel(gols, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Artilharia


### Assistências

In [8644]:
assistencias = df.copy()

In [8645]:
assistencias = (
    assistencias
      .groupby(["player_id", "name", "position", "team"], as_index=False)["goal_assist"]
      .sum()
      .rename(columns={"goal_assist": "assistencias"})
      .sort_values("assistencias", ascending=False)
      .reset_index(drop=True))

In [8646]:
assistencias = assistencias[assistencias["assistencias"] > 0]

In [8647]:
assistencias = assistencias.head(10)

In [8648]:
assistencias = eliminar_primeira_coluna(assistencias)

assistencias.head(3)

,name,position,team,assistencias
0,Leonardo Fernández,Meio-campo,Peñarol,4
1,Cecilio Domínguez,Meio-campo,Cerro Porteño,3
2,Marino Hinestroza,Meio-campo,Atlético Nacional,3


In [8649]:
aba = "Assistências"

salvar_df_no_excel(assistencias, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Assistências


### Participação em Gols

In [8650]:
participacao_gols = df.copy()

In [8651]:
participacao_gols = participacao_gols[["player_id", "name", "position", "team", "goals", "goal_assist"]]

participacao_gols.head(3)

,player_id,name,position,team,goals,goal_assist
0,34430,Raúl Olivares,Goleiro,Monagas,0,0
1,900191,Emanuel Iñíguez,Defensor,Monagas,0,0
2,933167,Oscar Piris,Defensor,Monagas,0,0


In [8652]:
participacao_gols["participacoes"] = (participacao_gols["goals"] + participacao_gols["goal_assist"])

participacao_gols.head(3)

,player_id,name,position,team,goals,goal_assist,participacoes
0,34430,Raúl Olivares,Goleiro,Monagas,0,0,0
1,900191,Emanuel Iñíguez,Defensor,Monagas,0,0,0
2,933167,Oscar Piris,Defensor,Monagas,0,0,0


In [8653]:
participacao_gols = (
    participacao_gols
        .groupby(["player_id", "name", "position", "team"], as_index=False)["participacoes"]
        .sum()
        .sort_values("participacoes", ascending=False)
        .reset_index(drop=True)
)

In [8654]:
participacao_gols = participacao_gols[participacao_gols["participacoes"] > 0]

In [8655]:
participacao_gols = participacao_gols.head(10)

In [8656]:
participacao_gols = eliminar_primeira_coluna(participacao_gols)

participacao_gols.head(3)

,name,position,team,participacoes
0,Adrián Martínez,Atacante,Racing,9
1,José López,Atacante,Palmeiras,8
2,Alan Patrick,Meio-campo,Internacional,7


In [8657]:
aba = "Participação em Gols"

salvar_df_no_excel(participacao_gols, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Participação em Gols


### Rating Total

In [8658]:
rating_total = df.copy()

In [8659]:
rating_total = (
    rating_total
        .groupby(["player_id", "name", "position", "team"], as_index=False)["rating"]
        .sum()
        .sort_values("rating", ascending=False)
        .reset_index(drop=True)
)

In [8660]:
rating_total = rating_total[rating_total["rating"] > 0]

rating_total.head(3)

,player_id,name,position,team,rating
0,331437,Guillermo Viscarra,Goleiro,Alianza Lima,84.9
1,876929,Kevin Quevedo,Atacante,Alianza Lima,83.9
2,31175,Hernán Barcos,Atacante,Alianza Lima,83.0


In [8661]:
rating_total = rating_total.head(10)

In [8662]:
rating_total = eliminar_primeira_coluna(rating_total)

rating_total.head(3)

,name,position,team,rating
0,Guillermo Viscarra,Goleiro,Alianza Lima,84.9
1,Kevin Quevedo,Atacante,Alianza Lima,83.9
2,Hernán Barcos,Atacante,Alianza Lima,83.0


In [8663]:
aba = "Rating Total"

salvar_df_no_excel(rating_total, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Rating Total


### Rating Médio

In [8664]:
rating_medio = df.copy()

Separando apenas os jogadores que atuaram em 10% do total de minutos

In [8665]:
minutagem = rating_medio.copy()

minutagem = minutagem[["player_id", "minutes_played"]]

In [8666]:
minutagem = (
    minutagem
        .groupby("player_id", as_index=False)["minutes_played"]
        .sum()
        .rename(columns={"minutes_played": "minutos"})
        .sort_values("minutos", ascending=False)
        .reset_index(drop=True)
)

In [8667]:
proporcao_minima = (minutagem["minutos"].max()) * 0.40

minutagem = minutagem[minutagem["minutos"] >= proporcao_minima]

minutagem.head(3)

,player_id,minutos
0,331437,1080
1,338957,1073
2,1020375,990


Fazendo o rating médio

In [8668]:
rating_medio = (
    rating_medio
        .groupby(["player_id", "name", "position", "team"], as_index=False)["rating"]
        .mean()
        .rename(columns={"rating": "rating_medio"})
        .sort_values("rating_medio", ascending=False)
        .reset_index(drop=True)
)

In [8669]:
rating_medio["rating_medio"] = rating_medio["rating_medio"].round(2)

rating_medio.head(3)

,player_id,name,position,team,rating_medio
0,876307,Ramiro Vaca,Meio-campo,Bolívar,8.45
1,885179,Paulinho,Atacante,Palmeiras,8.20
2,146416,Óscar Romero,Meio-campo,Internacional,8.20


In [8670]:
rating_medio = rating_medio.merge(minutagem, how="inner", on="player_id")

rating_medio.head(3)

,player_id,name,position,team,rating_medio,minutos
0,874964,Robert Rojas,Defensor,Libertad,7.75,696
1,1403559,Franco Mastantuono,Meio-campo,River Plate,7.73,484
2,928237,Santiago Sosa,Defensor,Racing,7.72,885


In [8671]:
rating_medio = rating_medio[["name", "position", "team", "rating_medio"]]

rating_medio.head(3)

,name,position,team,rating_medio
0,Robert Rojas,Defensor,Libertad,7.75
1,Franco Mastantuono,Meio-campo,River Plate,7.73
2,Santiago Sosa,Defensor,Racing,7.72


In [8672]:
rating_medio = rating_medio.head(10)

In [8673]:
aba = "Rating Médio"

salvar_df_no_excel(rating_medio, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Rating Médio


### Grandes Chances Criadas

In [8674]:
grandes_chances = df.copy()

In [8675]:
grandes_chances = (
    grandes_chances
        .groupby(["player_id", "name", "position", "team"], as_index=False)["big_chance_created"]
        .sum()
        .rename(columns={"big_chance_created": "grandes_chances"})
        .sort_values("grandes_chances", ascending=False)
        .reset_index(drop=True)
)

In [8676]:
grandes_chances = grandes_chances[grandes_chances["grandes_chances"] > 0]

In [8677]:
grandes_chances = grandes_chances.head(10)

In [8678]:
grandes_chances = eliminar_primeira_coluna(grandes_chances)

grandes_chances.head(3)

,name,position,team,grandes_chances
0,Gabriel Rojas,Meio-campo,Racing,7
1,Gastón Martirena,Meio-campo,Racing,5
2,Patricio Rodríguez,Atacante,Bolívar,5


In [8679]:
aba = "Grandes Chances"

salvar_df_no_excel(grandes_chances, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Grandes Chances


### Passes Chave

In [8680]:
passes_chave = df.copy()

In [8681]:
passes_chave = (
    passes_chave
        .groupby(["player_id", "name", "position", "team"], as_index=False)["key_pass"]
        .sum()
        .rename(columns={"key_pass": "passes_chave"})
        .sort_values("passes_chave", ascending=False)
        .reset_index(drop=True)
)

In [8682]:
passes_chave = passes_chave[passes_chave["passes_chave"] > 0]

passes_chave.head(3)

,player_id,name,position,team,passes_chave
0,1018468,Tiago Palacios,Meio-campo,Estudiantes,27
1,846411,Leonardo Fernández,Meio-campo,Peñarol,24
2,341065,Federico Carrizo,Meio-campo,Cerro Porteño,24


In [8683]:
passes_chave = passes_chave.head(10)

In [8684]:
passes_chave = eliminar_primeira_coluna(passes_chave)

passes_chave.head(3)

,name,position,team,passes_chave
0,Tiago Palacios,Meio-campo,Estudiantes,27
1,Leonardo Fernández,Meio-campo,Peñarol,24
2,Federico Carrizo,Meio-campo,Cerro Porteño,24


In [8685]:
aba = "Passes Chave"

salvar_df_no_excel(passes_chave, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: Passes Chave


### xG Total

In [8686]:
xg_total = df.copy()

In [8687]:
xg_total = (
    xg_total
        .groupby(["player_id", "name", "position", "team"], as_index=False)["expected_goals"]
        .sum()
        .rename(columns={"expected_goals": "xg_total"})
        .sort_values("xg_total", ascending=False)
        .reset_index(drop=True)
)

In [8688]:
xg_total = xg_total[xg_total["xg_total"] > 1]

xg_total.head(3)

,player_id,name,position,team,xg_total
0,906811,Adrián Martínez,Atacante,Racing,7.0037
1,31175,Hernán Barcos,Atacante,Alianza Lima,4.9790
2,358938,Sebastián Driussi,Atacante,River Plate,4.6593


In [8689]:
xg_total = xg_total.head(10)

In [8690]:
xg_total["xg_total"] = xg_total["xg_total"].round(2)

In [8691]:
xg_total = eliminar_primeira_coluna(xg_total)

In [8692]:
aba = "xG Total"

salvar_df_no_excel(xg_total, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: xG Total


### xG Médio

In [8693]:
xg_medio = df.copy()

In [8694]:
xg_medio = (
    xg_medio
    .groupby(["player_id", "name", "position", "team"], as_index=False)["expected_goals"]
    .mean()
    .rename(columns={"expected_goals": "xg_medio"})
    .sort_values("xg_medio", ascending=False)
    .reset_index(drop=True)
)

In [8695]:
xg_medio = xg_medio[xg_medio["xg_medio"] > 0]

xg_medio.head(3)

,player_id,name,position,team,xg_medio
0,906811,Adrián Martínez,Atacante,Racing,0.70037
1,1597265,Estêvão,Meio-campo,Palmeiras,0.66354
2,333367,Cecilio Domínguez,Meio-campo,Cerro Porteño,0.61580


In [8696]:
xg_medio = xg_medio.merge(minutagem, how="inner", on="player_id")

xg_medio.head(3)

,player_id,name,position,team,xg_medio,minutos
0,906811,Adrián Martínez,Atacante,Racing,0.70037,869
1,358938,Sebastián Driussi,Atacante,River Plate,0.51770,609
2,124997,Alan Patrick,Meio-campo,Internacional,0.50990,720


In [8697]:
xg_medio = xg_medio[["name", "position", "team", "xg_medio"]]

In [8698]:
xg_medio = xg_medio.head(10)

In [8699]:
xg_medio["xg_medio"] = xg_medio["xg_medio"].round(2)

In [8700]:
aba = "xG Médio"

salvar_df_no_excel(xg_medio, caminho, aba, index=False)

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_especial.xlsx | Aba: xG Médio


# Prompt

In [8701]:
texto = f"""[ PAPEL ]

Atue como Designer e Editor de Dados Sênior. Crie uma apresentação tipo e-book para a Bet Dados no Gamma (tema: Coal).

[ CONTEXTO ]

Público leigo–intermediário; foco em clareza.

Use somente os dados fornecidos. Sem inferências, estimativas ou previsões.

Se um dado não existir, omite a seção.

Duelo no formato “Time A x Time B” (nunca “vs”).

Estética “big tech”: precisão, consistência, espaço em branco e alinhamento milimétrico.

[ ESTILO GLOBAL — CONTRASTE OBRIGATÓRIO ] Forçar cor do texto = #FFFFFF para qualquer elemento textual, sem exceções:

Aplica cor #FFFFFF com opacidade 100% para: títulos, subtítulos, parágrafos, bullets, textos de cartões, subtítulos/captions, notas, footers, botões, placeholders, links, números de ordenação (ex.: “01”, “02” no canto), legendas de gráficos, rótulos de eixos, rótulos de dados, títulos de tabela e células de tabela.

Proibido usar cinzas/“muted”, opacidade < 100% ou “secondary text”. Nada de “texto desabilitado” em cinza.

Se houver fundo claro/Imagem, coloque placa/overlay escuro (ex.: #0A0A0A 90–95%) antes de posicionar o texto.

Ícones/linhas divisórias podem usar #FFFFFF a 70–90% de opacidade; o texto nunca.

[ GRÁFICOS ]

Desligar cores padrão do tema.

Barras/fatias/linhas: usar apenas cinzas claros #F2F2F2, #EAEAEA, #E0E0E0, #D5D5D5. (Nunca cinzas escuros.)

Contorno das barras/fatias: #FFFFFF (80–100% opacidade) para separação.

Legenda, eixos e labels: #FFFFFF.

Grade discreta: #9E9E9E a 25–35% opacidade. Fundo do gráfico transparente.

Cores oficiais dos clubes apenas como acento (borda/marker), mantendo contraste AA.

[ IDENTIDADE E IMAGENS ]

Escudos oficiais (sem distorção nem marca d’água). Se não houver, usar placeholder neutro.

Proibição absoluta de imagens com marca d’água.

Paleta do deck: #001636 como acento + neutros escuros de fundo + dados em cinzas claros.

[ TAREFA — SLIDES ] (omita os que não tiverem dados)

Capa — Título “[Time A x Time B] | Sub “[Data] — [Hora]”; chips de mandante/visitante e fase; troféu oficial como hero; overlay escuro se preciso.

Publicidade Bet Dados (obrigatório) — texto, CTA “Assinar agora” → [https://pay.cakto.com.br/35ww97h_563195](https://pay.cakto.com.br/35ww97h_563195) e placeholder de logo.

Ficha do Jogo — competição, temporada, data/hora, estádio/mandante.

Panorama das Equipes (cards lado a lado, 2–4 linhas por equipe, texto branco).

Indicadores-Chave (tabela/tiles).

Distribuição de Gols por Minutagem.

Ofensivo vs Defensivo.

Bolas Paradas.

Disciplina.

Tendências Temporais.

Head-to-Head (se houver).

Glossário Rápido (somente termos presentes).

Notas Metodológicas (origem/janela/limitações; “sem prognósticos”).

Slide Final (QR/CTA).

[ REGRAS DE CONTEÚDO ]

Sem previsões/odds.

Em hipótese alguma utilize imagens com marca d'água.

Nomes dos clubes exatamente como informados (acentos e caixa).

Percentuais com até 2 casas decimais; unidades PT-BR.

[ ENTRADAS ] Duelo, competição, temporada, data/hora e todas as estatísticas fornecidas (médias, totais, séries, rankings etc.).

[ AUDITORIA DE CONTRASTE (passo obrigatório no final) ]

Varra o deck e liste qualquer elemento cujo texto NÃO esteja #FFFFFF (inclui textos de cartões, subtítulos/captions, numeradores “01/02/03/04”, descrições, botões, rodapés e legendas).

Corrija automaticamente para #FFFFFF e reporte “Corrigido: [elementos]”. Se algo não puder ser alterado, substitua por placa/overlay escuro até garantir contraste AA.

[ CRITÉRIOS DE QUALIDADE ]

Fidelidade 100% aos dados.

Texto 100% branco em todo o deck; gráficos em cinzas claros; contraste AA mínimo.

Coesão (grid, margens, alinhamentos).

Didática (1 frase objetiva por gráfico).

Conformidade de marca (escudos/nome de clubes corretos; zero marca d’água).

[ CHECKLIST FINAL ]

Todo texto branco #FFFFFF (inclui cartões, captions e numeradores “01/02/03/…”).

Séries de gráficos somente #F2F2F2–#D5D5D5 + contorno branco; legenda/eixos brancos.

Sem imagens com marca d'água; escudos/taça oficiais.

“x” no duelo; nomes dos clubes fiéis.

Seções sem dados omitidas; nenhum número inventado.

Exportável para PDF com contraste perfeito.

Não trazer imagens com marca d'água.

Os dados e as estatísticas são de um levantamento da Bet Dados.

NÃO precisa informar a temporada.

Reforçando: NÃO FAÇA NENHUMA PREVISÃO PARA A PARTIDA!

Nos gráficos, SEMPRE ative o rótulo de dados (mostrar valores).

Importante: a Bet Dados NÃO oferece dados em tempo real. Não coloque isso na apresentação. Não coloque NENHUM ponto negativo da Bet Dados.

[ COMPETIÇÃO ANALISADA]

Competição: {campeonato}

Temporada: {temporada}


Segue abaixo as estatísticas para a apresentação:

Temporada atual

{tabela}



As informações abaixo trazem o top 10 de cada dado. Deixe isso explícito e coloque um asterisco em todos os slides informando que é apenas o top 10.

Mais Vitórias

{mais_vitorias}


Menos Vitórias

{menos_vitorias}


Mais Derrotas

{mais_derrotas}


Menos Derrotas

{menos_derrotas}


Maior Média de Pontos por Jogo como Mandante

{maior_media_ppj_casa}


Menor Média de Pontos por Jogo como Mandante

{menor_media_ppj_casa}


Maior Média de Pontos por Jogo como Visitante

{maior_media_ppj_fora}


Menor Média de Pontos por Jogo como Visitante

{menor_media_ppj_fora}


Maior Média de Pontos por Jogo (Geral)

{maior_media_ppj}


Menor Média de Pontos por Jogo (Geral)

{menor_media_ppj}


Melhor Ataque

{melhor_ataque}


Maior Média de Gols Marcados

{melhor_media_gols_marcados}


Pior Ataque

{pior_ataque}


Menor Média de Gols Marcados

{pior_media_gm}


Melhor Defesa

{melhor_defesa}


Melhor Média de Gols Sofridos

{melhor_media_gs}


Pior Defesa

{pior_defesa}


Pior Média de Gols Sofridos

{pior_media_gs}


Mais Clean Sheets

{mais_clean_sheets}


Menor Clean Sheets

{menos_clean_sheets}


Menos Clean Goals

{menos_clean_goals}


Mais Clean Goals

{mais_clean_goals}


Maior Sequência sem Derrotas

{maior_invencibilidade}


Menor Sequência sem Derrotas

{menor_invencibilidade}


Invencibilidade Atual

{invencibilidade_atual}


Maior Sequência de Vitórias

{maximo_historico}


Menor Sequência de Vitórias

{minimo_historico}


Sequência Atual de Vitórias

{sequencia_vitorias_atual}


Posse de Bola Média

{posse_bola_media}


Maior xG Médio

{maior_xg_medio}


Menor xG Médio

{menor_xg_medio}


Maior Média de Finalizações

{maior_media_finalizacoes}


Maior Média de Finalizações Certas

{maior_media_finalizacoes_certas}


Aproveitamento das Finalizações

{aproveitamento_finalizacoes}


Média de Finalizações Dentro da Área

{media_finalizacao_dentro_area}


Entradas no Terço Final

{entradas_terco_final}


Grandes Chances Criadas

{grandes_chances}


Aproveitamento das Grandes Chances

{aproveitamento_grandes_chances}


Defesas do Goleiro

{defesas_goleiro}


Aproveitamento nos Passes

{aproveitamento_passes}


JOGADORES

Minutos Goleiros

{mais_minutos_goleiros}


Minutos Defensores

{mais_minutos_defensores}


Minutos Meias

{mais_minutos_meias}


Minutos Atacantes

{mais_minutos_atacantes}


Artilharia

{gols}


Assistências

{assistencias}


Participação em Gols

{participacao_gols}


Rating Total

{rating_total}


Rating Médio

{rating_medio}


Grandes Chances

{grandes_chances}


Passes Chave

{passes_chave}


xG Total

{xg_total}


xG Médio

{xg_medio}
"""

In [8702]:
from docx import Document

# ======== CAMINHO PARA ÁREA DE TRABALHO ========
caminho_arquivo = os.path.join(os.path.expanduser("~"), "Desktop", "prompt.docx")

# ======== CRIA O DOCUMENTO WORD ========
documento = Document()
documento.add_paragraph(texto)

# ======== SALVA ========
documento.save(caminho_arquivo)

print(f"Arquivo salvo com sucesso em: {caminho_arquivo}")

Arquivo salvo com sucesso em: C:\Users\stefa\Desktop\prompt.docx


# Finalizado

In [8703]:
print("Script finalizado.")

Script finalizado.
